Homework: Evaluation and Monitoring
In this homework, we'll evaluate the quality of our RAG system.

It's possible that your answers won't match exactly. If it's the case, select the closest one.

Solution:

Video: TBA
Notebook: TBA
Getting the data
Let's start by getting the dataset. We will use the data we generated in the module.

In particular, we'll evaluate the quality of our RAG system with gpt-4o-mini

Read it:

url = f'{github_url}?raw=1'
df = pd.read_csv(url)
We will use only the first 300 documents:

df = df.iloc[:300]
## Q1. Getting the embeddings model
Now, get the embeddings model multi-qa-mpnet-base-dot-v1 from the Sentence Transformer library

Note: this is not the same model as in HW3

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
Create the embeddings for the first LLM answer:

answer_llm = df.iloc[0].answer_llm
What's the first value of the resulting vector?

-0.42
-0.22
-0.02
0.21

In [46]:
import pandas as pd

github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [47]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [6]:
! pip install -q sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 17.8 MB/s eta 0:00:00


In [7]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [48]:
answer_llm = df.iloc[0].answer_llm

In [49]:
embeds = embedding_model.encode(answer_llm)
embeds[0]

-0.42244658

### A1.

**-0.42**

## Q2. Computing the dot product
Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the evaluations list

What's the 75% percentile of the score?

21.67
31.67
41.67
51.67

In [51]:
import numpy as np
from tqdm.auto import tqdm

# List to hold the scores
evaluations = []

# Compute dot product for all records
for rec in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = rec[1]['answer_llm']
    answer_orig = rec[1]['answer_orig']

    # Create embeddings
    emb_llm = embedding_model.encode(answer_llm)
    emb_orig = embedding_model.encode(answer_orig)

    # Compute dot product
    scores = np.dot(emb_llm, emb_orig)
    evaluations.append(scores)

# Calculate the 75th percentile of the scores
percentile_75 = np.percentile(evaluations, 75)
print(f"75th percentile of the scores: {percentile_75}")

  0%|          | 0/300 [00:00<?, ?it/s]

75th percentile of the scores: 31.674307823181152


### A2.

**31.67**

## Q3. Computing the cosine
From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

So we need to normalize them.

To do it, we

Compute the norm of a vector
Divide each element by this norm
So, for vector v, it'll be v / ||v||

In numpy, this is how you do it:

norm = np.sqrt((v * v).sum())
v_norm = v / norm
Let's put it into a function and then compute dot product between normalized vectors. This will give us cosine similarity

What's the 75% cosine in the scores?

0.63
0.73
0.83
0.93

In [52]:
# Function to normalize a vector
def normalize(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

# List to hold the normalized scores
normalized_evaluations = []

# Compute cosine similarity for all records
for rec in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = rec[1]['answer_llm']
    answer_orig = rec[1]['answer_orig']

    # Create embeddings
    emb_llm = embedding_model.encode(answer_llm)
    emb_orig = embedding_model.encode(answer_orig)

    # Normalize embeddings
    emb_llm_norm = normalize(emb_llm)
    emb_orig_norm = normalize(emb_orig)

    # Compute cosine similarity
    cosine_similarity = np.dot(emb_llm_norm, emb_orig_norm)
    normalized_evaluations.append(cosine_similarity)

# Calculate the 75th percentile of the normalized scores
normalized_percentile_75 = np.percentile(normalized_evaluations, 75)
print(f"75th percentile of the normalized scores: {normalized_percentile_75}")

  0%|          | 0/300 [00:00<?, ?it/s]

75th percentile of the normalized scores: 0.8362347930669785


### A3.

**0.83**

## Q4. Rouge
Now we will explore an alternative metric - the ROUGE score.

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

We don't need to implement it ourselves, there's a python package for it:

pip install rouge
(The latest version at the moment of writing is 1.0.1)

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)

from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

rouge-1 - the overlap of unigrams,
rouge-2 - bigrams,
rouge-l - the longest common subsequence
What's the F score for rouge-1?

0.35
0.45
0.55
0.65

In [26]:
! pip install -qU rouge

In [53]:
from rouge import Rouge

# Initialize ROUGE scorer
rouge_scorer = Rouge()

# Get the answers at index 10
index_10 = 10
r = df.iloc[index_10]

# Compute ROUGE scores
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
rouge_1_f = scores['rouge-1']['f']
print(f"ROUGE-1 F score: {rouge_1_f}")

ROUGE-1 F score: 0.45454544954545456


### A4.

**0.45**

## Q5. Average rouge score
Let's compute the average between rouge-1, rouge-2 and rouge-l for the same record from Q4

0.35
0.45
0.55
0.65

In [54]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
print(f"Average ROUGE score: {rouge_avg}")

Average ROUGE score: 0.35490034990035496


### A5.

**0.35**

## Q6. Average rouge score for all the data points
Now let's compute the score for all the records

rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
And create a dataframe from them

What's the agerage rouge_2 across all the records?

0.10
0.20
0.30
0.40

In [55]:
# Lists to hold ROUGE scores
rouge_2_scores = []

# Compute ROUGE scores for all records
for rec in tqdm(df.iterrows(), total=df.shape[0]):
    answer_llm = rec[1]['answer_llm']
    answer_orig = rec[1]['answer_orig']

    # Compute ROUGE scores
    scores = rouge_scorer.get_scores(answer_llm, answer_orig)[0]

    # Extract ROUGE-2 F1 score
    rouge_2_f1 = scores['rouge-2']['f']
    rouge_2_scores.append(rouge_2_f1)

# Calculate the average ROUGE-2 F1 score
average_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
print(f"Average ROUGE-2 F1 Score: {average_rouge_2}")

  0%|          | 0/300 [00:00<?, ?it/s]

Average ROUGE-2 F1 Score: 0.20696501983423318


In [1]:
# Lists to hold ROUGE scores
rouge_2_scores = []

# Compute ROUGE scores for all records
for index, row in df.iterrows():
    answer_llm = row['answer_llm']
    answer_orig = row['answer_orig']

    # Compute ROUGE scores
    scores = rouge_scorer.get_scores(answer_llm, answer_orig)[0]

    # Extract ROUGE-2 F1 score
    rouge_2_f1 = scores['rouge-2']['f']
    rouge_2_scores.append(rouge_2_f1)

# Calculate the average ROUGE-2 F1 score
average_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)

print(f"Average ROUGE-2 F1 Score: {average_rouge_2}")

NameError: name 'df' is not defined

### A6.

**0.20**